# VTK - Output

Netgen/NGSolve comes with several convenient ways to visualize your PDE solutions: the webgui, the legacy netgen GUI, ... For some use cases visualization toolboxes like [ParaView](https://www.paraview.org/) may still be able to obtain more insight or more complex visualizations from you computational results.

To put your results in a corresponding format you can use the `VTKOutput` of NGSolve. 

A `VTKOutput` object obtains a list of coefficient functions (scalar or vector) and a list of labels. Together with the mesh information these information will be put into a vtk-formatted file on every `Do`-call of the object. As VTK essentially only draws piecewise linears you may want to describe a `subdivision` argument larger `0` to obtain higher resolution of a higher order FE function.

Let's explore a few use cases by examples:

## Example 1: single solution output
Let's recap the Poisson solution from [unit 1.1](../unit-1.1-poisson/poisson.ipynb):

In [ ]:
from ngsolve import *
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
fes = H1(mesh, order=2, dirichlet="bottom|right")
gfu = GridFunction(fes) 
a = BilinearForm(fes, symmetric=True)
a += grad(fes.TrialFunction())*grad(fes.TestFunction())*dx
a.Assemble()
f = LinearForm(fes)
f += x*fes.TestFunction()*dx
f.Assemble()
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec

Now, instead of using the webgui, let us export the result data to a vtk file:

In [ ]:
vtk = VTKOutput(mesh,coefs=[gfu],names=["sol"],filename="vtk_example1",subdivision=2)
vtk.Do()

Note, only the `Do()`-call writes the output. Let's see what we got:

In [ ]:
!ls -lh vtk_example1*

We obtained the file `vtk_example1.vtu`. This file can be opened with ParaView, e.g. with `paraview vtk_example1.vtu` where you can select your solution field and use several "filters" to process the data to get nice visualizations as the following:
<div>
<img src="vtk_example1.png" width="600"/>
</div>
You can now play around with tools like paraview to do many different complex operations.

## Example 2: Series of data

In some cases you may want to analyse a sequence of simulation, e.g. for generating a video. In this case, you simply call `Do()` several times. You can also provide an additional time stamp with e.g. `Do(time=0.3)`.

Let's take a time-dependent coefficient function and put this to a sequence of VTK outputs:

In [ ]:
t = Parameter(0)
coef = sin(4*(x+y+t))
vtkout = VTKOutput(mesh,coefs=[coef],names=["sol"],filename="vtk_example2",subdivision=2)
vtkout.Do(time = 0.0)
for i in range(20):
    t.Set((i+1)/20)
    vtkout.Do(time = (i+1)/20.0)

Let's see what we got this time:

In [ ]:
ls -lh vtk_example2*


* We obtained 21 `.vtu`-files corresponding to the 21 `Do`-calls.
* Only the first `Do`-call does not have an additional suffix, while later calls append the `_step...`-suffix. 
* Furthermore there is a `.pvd` file which bundles the individual files and associates them to the provided time stamps (if provided):

In [ ]:
!cat vtk_example2.pvd

Now you can open this "meta file" with paraview and visualize the time series with many different filter. Finally, you can export stills or videos from that. It will look like this:
<div>
<img src="vtk_example2.png" width="600"/>
</div>

## Further options

To learn more about possible options you show call `help(VTKOutput)`. Nevertheless, let us comment on a few of theme:

* `VTKOutput(..., subdivision=i)` refines the mesh (locally and only temporarily) as often as `i`. Note that when opened with ParaView the visualized mesh no longer coincides with your computational mesh.
* `VTKOutput(..., only_element=i)` puts only the data of one selected element out. This can be interesting for basis function visualization or alike.
* `VTKOutput(..., floatsize=fs)` where `fs` is either `"single"` or `"double"` decides on the precision of the output. Default is `"double"`. Using `"single"` can significantly reduce the used disk space.

there are further options for the `Do(...)`-call that we briefly discuss:

* `....Do(time=t)` adds the time stamp `t` to the meta-file if a series of outputs is used.
* `....Do(vb=vb)` allows you to switch from volume mesh output (`vb=VOL`, default) to surface mesh output (`vb=BND`)
* `....Do(drawelems=els)` allows you to only write a submesh of the underlying mesh where the corresponding set of elements is prescribed by the `BitArray` `els`.


In [ ]:
help(VTKOutput)